The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U mxnet-cu101==1.7.0


# Gerenciamento de Parâmetros

Depois de escolher uma arquitetura
e definir nossos hiperparâmetros,
passamos para o ciclo de treinamento,
onde nosso objetivo é encontrar valores de parâmetro
que minimizam nossa função de perda.
Após o treinamento, precisaremos desses parâmetros
para fazer previsões futuras.
Além disso, às vezes desejamos
para extrair os parâmetros
seja para reutilizá-los em algum outro contexto,
para salvar nosso modelo em disco para que
pode ser executado em outro *software*,
ou para exame na esperança de
ganhar compreensão científica.

Na maioria das vezes, seremos capazes de
ignorar os detalhes essenciais
de como os parâmetros são declarados
e manipulado, contando com estruturas de *Deep Learning*
para fazer o trabalho pesado.
No entanto, quando nos afastamos de
arquiteturas empilhadas com camadas padrão,
às vezes precisaremos 
declarar e manipular parâmetros.
Nesta seção, cobrimos o seguinte:

* Parâmetros de acesso para depuração, diagnóstico e visualizações.
* Inicialização de parâmetros.
* Parâmetros de compartilhamento em diferentes componentes do modelo.

Começamos nos concentrando em um MLP com uma camada oculta.


In [1]:
from mxnet import init, np, npx
from mxnet.gluon import nn

npx.set_np()

net = nn.Sequential()
net.add(nn.Dense(8, activation='relu'))
net.add(nn.Dense(1))
net.initialize()  # Use o método de inicialização padrão

X = np.random.uniform(size=(2, 4))
net(X)  # Forward computation

array([[0.0054572 ],
       [0.00488594]])

## Acesso a Parâmetros

Vamos começar explicando como acessar os parâmetros
dos modelos que você já conhece.
Quando um modelo é definido por meio da classe `Sequential`,
podemos primeiro acessar qualquer camada indexando
no modelo como se fosse uma lista.
Os parâmetros de cada camada são convenientemente
localizado em seu atributo.
Podemos inspecionar os parâmetros da segunda camada totalmente conectada da seguinte maneira.


In [2]:
print(net[1].params)

dense1_ (
  Parameter dense1_weight (shape=(1, 8), dtype=float32)
  Parameter dense1_bias (shape=(1,), dtype=float32)
)


A saída nos diz algumas coisas importantes.
Primeiro, esta camada totalmente conectada
contém dois parâmetros,
correspondendo aos
pesos e vieses, respectivamente.
Ambos são armazenados como *floats* de precisão simples (float32).
Observe que os nomes dos parâmetros
nos permitem identificar de forma única
parâmetros de cada camada,
mesmo em uma rede contendo centenas de camadas.


### Parâmetros Direcionados

Observe que cada parâmetro é representado
como uma instância da classe de parâmetro.
Para fazer algo útil com os parâmetros,
primeiro precisamos acessar os valores numéricos subjacentes.
Existem várias maneiras de fazer isso.
Alguns são mais simples, enquanto outros são mais gerais.
O código a seguir extrai o viés
da segunda camada de rede neural, que retorna uma instância de classe de parâmetro, e
acessa posteriormente o valor desse parâmetro.


In [3]:
print(type(net[1].bias))
print(net[1].bias)
print(net[1].bias.data())

<class 'mxnet.gluon.parameter.Parameter'>
Parameter dense1_bias (shape=(1,), dtype=float32)
[0.]


Os parâmetros são objetos complexos,
contendo valores, gradientes,
e informações adicionais.
É por isso que precisamos solicitar o valor explicitamente.

Além do valor, cada parâmetro também nos permite acessar o gradiente. Como ainda não invocamos a *backpropagation* para esta rede, ela está em seu estado inicial.


In [4]:
net[1].weight.grad()

array([[0., 0., 0., 0., 0., 0., 0., 0.]])

### Todos os Parâmetros de Uma Vez

Quando precisamos realizar operações em todos os parâmetros,
acessá-los um por um pode se tornar tedioso.
A situação pode ficar especialmente complicada
quando trabalhamos com blocos mais complexos (por exemplo, blocos aninhados),
uma vez que precisaríamos voltar recursivamente
através de toda a árvore para extrair
parâmetros de cada sub-bloco. Abaixo, demonstramos como acessar os parâmetros da primeira camada totalmente conectada versus acessar todas as camadas.


In [5]:
print(net[0].collect_params())
print(net.collect_params())

dense0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
)
sequential0_ (
  Parameter dense0_weight (shape=(8, 4), dtype=float32)
  Parameter dense0_bias (shape=(8,), dtype=float32)
  Parameter dense1_weight (shape=(1, 8), dtype=float32)
  Parameter dense1_bias (shape=(1,), dtype=float32)
)


Isso nos fornece outra maneira de acessar os parâmetros da rede como segue.


In [6]:
net.collect_params()['dense1_bias'].data()

array([0.])

### Coletando Parâmetros de Blocos Aninhados

Vamos ver como funcionam as convenções de nomenclatura de parâmetros
se aninharmos vários blocos uns dentro dos outros.
Para isso, primeiro definimos uma função que produz blocos
(uma fábrica de blocos, por assim dizer) e então
combine-os dentro de blocos ainda maiores.


In [7]:
def block1():
    net = nn.Sequential()
    net.add(nn.Dense(32, activation='relu'))
    net.add(nn.Dense(16, activation='relu'))
    return net

def block2():
    net = nn.Sequential()
    for _ in range(4):
        # Nested here
        net.add(block1())
    return net

rgnet = nn.Sequential()
rgnet.add(block2())
rgnet.add(nn.Dense(10))
rgnet.initialize()
rgnet(X)

array([[-6.3465846e-09, -1.1096752e-09,  6.4161787e-09,  6.6354140e-09,
        -1.1265507e-09,  1.3284951e-10,  9.3619388e-09,  3.2229084e-09,
         5.9429879e-09,  8.8181435e-09],
       [-8.6219423e-09, -7.5150686e-10,  8.3133251e-09,  8.9321128e-09,
        -1.6740003e-09,  3.2405989e-10,  1.2115976e-08,  4.4926449e-09,
         8.0741742e-09,  1.2075874e-08]])

Agora que projetamos a rede,
vamos ver como está organizado.


In [8]:
print(rgnet.collect_params)
print(rgnet.collect_params())

<bound method Block.collect_params of Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Dense(4 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (1): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (2): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
    (3): Sequential(
      (0): Dense(16 -> 32, Activation(relu))
      (1): Dense(32 -> 16, Activation(relu))
    )
  )
  (1): Dense(16 -> 10, linear)
)>
sequential1_ (
  Parameter dense2_weight (shape=(32, 4), dtype=float32)
  Parameter dense2_bias (shape=(32,), dtype=float32)
  Parameter dense3_weight (shape=(16, 32), dtype=float32)
  Parameter dense3_bias (shape=(16,), dtype=float32)
  Parameter dense4_weight (shape=(32, 16), dtype=float32)
  Parameter dense4_bias (shape=(32,), dtype=float32)
  Parameter dense5_weight (shape=(16, 32), dtype=float32)
  Parameter dense5_bi

Uma vez que as camadas são aninhadas hierarquicamente,
também podemos acessá-los como se
indexação por meio de listas aninhadas.
Por exemplo, podemos acessar o primeiro bloco principal,
dentro dele o segundo sub-bloco,
e dentro disso o viés da primeira camada,
com o seguinte.


In [9]:
rgnet[0][1][0].bias.data()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Inicialização de Parâmetros

Agora que sabemos como acessar os parâmetros,
vamos ver como inicializá-los corretamente.
Discutimos a necessidade de inicialização adequada em :numref:`sec_numerical_stability`.
A estrutura de *Deep Learning* fornece inicializações aleatórias padrão para suas camadas.
No entanto, muitas vezes queremos inicializar nossos pesos
de acordo com vários outros protocolos. A estrutura fornece mais comumente
protocolos usados e também permite criar um inicializador personalizado.


Por padrão, MXNet inicializa os parâmetros de peso ao desenhar aleatoriamente de uma distribuição uniforme $U(-0.07, 0.07)$,
limpar os parâmetros de polarização para zero.
O módulo `init` do MXNet oferece uma variedade
de métodos de inicialização predefinidos.


### Inicialização *Built-in* 

Vamos começar chamando inicializadores integrados.
O código abaixo inicializa todos os parâmetros de peso
como variáveis aleatórias gaussianas
com desvio padrão de 0,01, enquanto os parâmetros de polarização são zerados.


In [10]:
# Aqui `force_reinit` garante que os parâmetros são inciados mesmo se
# eles já foram iniciados anteriormente
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0]

array([-0.00324057, -0.00895028, -0.00698632,  0.01030831])

Também podemos inicializar todos os parâmetros
a um determinado valor constante (digamos, 1).


In [11]:
net.initialize(init=init.Constant(1), force_reinit=True)
net[0].weight.data()[0]

array([1., 1., 1., 1.])

Também podemos aplicar inicializadores diferentes para certos blocos.
Por exemplo, abaixo inicializamos a primeira camada
com o inicializador Xavier
e inicializar a segunda camada
para um valor constante de 42.


In [12]:
net[0].weight.initialize(init=init.Xavier(), force_reinit=True)
net[1].initialize(init=init.Constant(42), force_reinit=True)
print(net[0].weight.data()[0])
print(net[1].weight.data())

[-0.17594433  0.02314097 -0.1992535   0.09509248]
[[42. 42. 42. 42. 42. 42. 42. 42.]]


### Inicialização Customizada

Às vezes, os métodos de inicialização de que precisamos
não são fornecidos pela estrutura de *Deep Learning*.
No exemplo abaixo, definimos um inicializador
para qualquer parâmetro de peso $w$ usando a seguinte distribuição estranha:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Aqui definimos uma subclasse da classe `Initializer`.
Normalmente, só precisamos implementar a função `_init_weight`
que leva um argumento tensor (`data`)
e atribui a ele os valores inicializados desejados.


In [13]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        data[:] = np.random.uniform(-10, 10, data.shape)
        data *= np.abs(data) >= 5

net.initialize(MyInit(), force_reinit=True)
net[0].weight.data()[:2]

Init dense0_weight (8, 4)
Init dense1_weight (1, 8)


array([[ 0.       , -0.       , -0.       ,  8.522827 ],
       [ 0.       , -8.828651 , -0.       , -5.6012006]])

Observe que sempre temos a opção
de definir parâmetros diretamente.


In [14]:
net[0].weight.data()[:] += 1
net[0].weight.data()[0, 0] = 42
net[0].weight.data()[0]

array([42.      ,  1.      ,  1.      ,  9.522827])

Uma observação para usuários avançados:
se você quiser ajustar os parâmetros dentro de um escopo `autograd`,
você precisa usar `set_data` para evitar confundir
a mecânica de diferenciação automática.


## Parâmetros *Tied*


Frequentemente, queremos compartilhar parâmetros em várias camadas.
Vamos ver como fazer isso com elegância.
A seguir, alocamos uma camada densa
e usar seus parâmetros especificamente
para definir os de outra camada.


In [15]:
net = nn.Sequential()
# Precisamos dar as camadas compartilhadas um nome
# para que possamos referenciar seus parâmetros
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

X = np.random.uniform(size=(2, 20))
net(X)

# Checar se são os mesmos parâmetros
print(net[1].weight.data()[0] == net[2].weight.data()[0])
net[1].weight.data()[0, 0] = 100
# Garantindo que são o mesmo objeto ao invés de ter
# apenas o mesmo valor

print(net[1].weight.data()[0] == net[2].weight.data()[0])

[ True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True]


Este exemplo mostra que os parâmetros
da segunda e terceira camadas são amarrados.
Eles não são apenas iguais, eles são
representado pelo mesmo tensor exato.
Assim, se mudarmos um dos parâmetros,
o outro também muda.
Você pode se perguntar,
quando os parâmetros são amarrados
o que acontece com os gradientes?
Uma vez que os parâmetros do modelo contêm gradientes,
os gradientes da segunda camada oculta
e a terceira camada oculta são adicionadas juntas
durante a retropropagação.


## Sumário

* Temos várias maneiras de acessar, inicializar e vincular os parâmetros do modelo.
* Podemos usar inicialização personalizada.


## Exercícios

1. Use o modelo `FancyMLP` definido em :numref:`sec_model_construction` e acesse os parâmetros das várias camadas.
1. Observe o documento do módulo de inicialização para explorar diferentes inicializadores.
1. Construa um MLP contendo uma camada de parâmetros compartilhados e treine-o. Durante o processo de treinamento, observe os parâmetros do modelo e gradientes de cada camada.
1. Por que compartilhar parâmetros é uma boa ideia?


[Discussão](https://discuss.d2l.ai/t/56)


<!--stackedit_data:
eyJoaXN0b3J5IjpbMjI3MjUyNjYwLC03MTk0NTg0ODEsMjUwMT
g4NTk3LC0xNTkzNzg3NzI4LDE2NDI5Nzg1MDFdfQ==
-->
